In [1]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
sys.path.append('..')

import utilities.functions as functions


from utilities.functions import (
    summary,
    matriz_migracao_n,
    save_parquet,
    gerar_resumo_decis,
    resumo_coorte_ativa,
   merge_df

)

In [2]:
df_pub_u = pd.read_parquet("dados/gold/df_pub_un.parquet")

Sumarizacao na visao cliente - mes

In [3]:
summary(df_pub_u)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
customer_id,9387,4998,27aa088f786b252fd7add45dbd0ae11f36aad9f65d40ec...,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
origin_platform,9387,4,ANDROID,3920,NaN,NaN,NaN,NaN,NaN,NaN,NaN
is_target,9387,2,target,5350,NaN,NaN,NaN,NaN,NaN,NaN,NaN
order_created_month,9387.0,NaN,NaN,NaN,5.667412,5.436906,1.0,1.0,1.0,12.0,12.0
num_pedidos_mes,9387.0,NaN,NaN,NaN,3.584212,4.004847,1.0,1.0,2.0,4.0,62.0
num_pedidos_hist,9387.0,NaN,NaN,NaN,6.868222,7.25463,1.0,2.0,5.0,8.0,95.0
total_amount_mes,9387.0,NaN,NaN,NaN,171.77272,232.575469,3.9,48.935,98.0,200.675,4359.98
ticket_medio,9387.0,NaN,NaN,NaN,47.661578,28.913863,3.9,29.9,41.25,57.0,483.9


Clientes mes x target

In [4]:
clientes_mes = (df_pub_u.groupby(['order_created_month', 'is_target'])
              ['customer_id']
              .nunique()
              .reset_index(name='numero_clientes_distintos'))
clientes_mes

,order_created_month,is_target,numero_clientes_distintos
0,1,control,2180
1,1,target,2818
2,12,control,1630
3,12,target,2157


 Matriz de migracao - cliente

In [5]:
matriz_migracao(df_pub_u,mes_0=12,mes_1=1)

NameError: name 'matriz_migracao' is not defined

Dado o montante de clientes novos em janeiro a analise sera feita basead em cliente ambos os meses e cliente somente em janeiro

In [6]:
id_both_monht=df_pub_u[df_pub_u['order_created_month']==12]['customer_id'].unique()
publico_janeiro_dezembro = df_pub_u[df_pub_u['customer_id'].isin(id_both_monht)].reset_index(drop=True)
publico_janeiro = df_pub_u[~df_pub_u['customer_id'].isin(id_both_monht)].reset_index(drop=True)

In [ ]:
publico_janeiro.head()

In [ ]:
save_parquet(publico_janeiro_dezembro, "gold", "publico_janeiro_dezembro.parquet")


Pedidos por mes - target

In [ ]:
pedidos_total = (
    publico_janeiro_dezembro.groupby(['is_target','order_created_month'])['num_pedidos_mes']
      .sum()
      .reset_index(name='numero_de_pedidos_total')
)

pedidos_total

Distruibuicao do total amount por mes

In [ ]:
df_stats_mes = publico_janeiro_dezembro.groupby(['order_created_month', 'is_target'])['total_amount_mes'].agg(
    Média=('mean'),
    Mediana=('median'),
    Mínimo=('min'),
    Máximo=('max'),
    Desvio_Padrão=('std')
).round(2)
df_stats_mes

In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(
    data=publico_janeiro_dezembro,
    y='total_amount_mes',
    x='order_created_month',  
    hue='is_target',          
    
)

plt.title('Distribuição do Valor Gasto por Mês e Grupo (Target vs Control)')
plt.ylabel('Total Amount (R$)')
plt.xlabel('Mês')
plt.legend(title='Grupo')
plt.show()

Observa-se grande numero de outliers, verificar a distribuicao removendo outliers.

In [ ]:
publico_janeiro_dezembro['total_amount_mes'].describe().round(2)


In [ ]:
red_square = dict(markerfacecolor='r', markeredgecolor='r', marker='.')
publico_janeiro_dezembro['total_amount_mes'].plot(kind='box', xlim=(0, 500), vert=False, flierprops=red_square, figsize=(16,2))

In [ ]:
distance = 1.5 * (np.nanpercentile(publico_janeiro_dezembro['total_amount_mes'], 75) - np.nanpercentile(publico_janeiro_dezembro['total_amount_mes'], 25))
lim_sup=distance + np.nanpercentile(publico_janeiro_dezembro['total_amount_mes'], 75)

In [ ]:
pb_drop = publico_janeiro_dezembro.drop(
    publico_janeiro_dezembro[
        (publico_janeiro_dezembro['total_amount_mes'] == 0) |
        (publico_janeiro_dezembro['total_amount_mes'] > lim_sup)
    ].index,
    axis=0
)

In [ ]:
pb_drop['total_amount_mes'].describe().round(2)

In [ ]:
red_square = dict(markerfacecolor='r', markeredgecolor='r', marker='.')
pb_drop['total_amount_mes'].plot(kind='box', xlim=(0, 400), vert=False, flierprops=red_square, figsize=(16,2))

Decil corrigido

In [7]:
import pandas as pd

# Calcular cortes na base1 e aplicar na base2
cortes = pd.qcut(publico_janeiro_dezembro[publico_janeiro_dezembro['order_created_month']==12]['total_amount_mes'], q=10, retbins=True)[1]
publico_janeiro_dezembro['decil'] = pd.cut(publico_janeiro_dezembro['total_amount_mes'], bins=cortes, labels=range(1, 11))
print(publico_janeiro_dezembro[['total_amount_mes', 'decil']])

      total_amount_mes decil
0               160.00     8
1               260.99     9
2               152.50     8
3               326.40    10
4               164.09     8
...                ...   ...
8141            120.00     7
8142             74.50     5
8143            349.03    10
8144            136.50     7
8145            136.50     7

[8146 rows x 2 columns]


In [16]:
mes_1=1
mes_0=12
keep_from_df1 = ['customer_id', 'is_target', 'decil','total_amount_mes']  
keep_from_df2 = ['customer_id', 'decil','total_amount_mes']
df_decil=merge_df(publico_janeiro_dezembro[publico_janeiro_dezembro['order_created_month']==12][keep_from_df1].rename(columns={'decil': f'decil_m{mes_0}',
                                                                                                                      'total_amount_mes':f'total_amount_m{mes_0}'}), 
         publico_janeiro_dezembro[publico_janeiro_dezembro['order_created_month']==1][keep_from_df2].rename(columns={'decil': f'decil_m{mes_1}',
                                                                                                                    'total_amount_mes':f'total_amount_m{mes_1}'}), key='customer_id', how='left')
 

In [30]:
clientes_por_decil_target = (
        publico_janeiro_dezembro
        .groupby(['order_created_month','decil', 'is_target'],observed=False)
        .agg(
            total_clientes=('customer_id', 'nunique'),
            avg_amount=('total_amount_mes', 'mean')
            #avg_amount_1=('total_amount_m1', 'mean')
            #avg_pedidos_12=('num_pedidos_mes_12', 'mean'),
            #avg_pedidos_1=('num_pedidos_mes_1', 'mean')
        )
        .reset_index()
    )

In [34]:
matriz_migracao_n(publico_janeiro_dezembro, mes_0=12, mes_1=1, group_by_extra='decil')

/Users/maceli/ifood_case/code/notebook/../utilities/functions.py:1878: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  .pivot_table(index=index_cols, columns='order_created_month', values='presenca', fill_value=0)
/Users/maceli/ifood_case/code/notebook/../utilities/functions.py:1891: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return (clientes_temp.groupby(group_by_cols).size()


,mes_12,mes_1,is_target,decil,total_clientes
0,0,0,control,1,0
1,0,0,control,2,0
2,0,0,control,3,0
3,0,0,control,4,0
4,0,0,control,5,0
...,...,...,...,...,...
75,1,1,target,6,74
76,1,1,target,7,49
77,1,1,target,8,62
78,1,1,target,9,67


In [37]:
df_decil.head()

,customer_id,is_target,decil_m12,total_amount_m12,decil_m1,total_amount_m1
0,fff0bcf899914ef1a4441277ca547b0282750344e95090...,target,8,160.00,9,260.99
1,fff02e65f78e3377668f6d87e6a7ff8f0ee0f18e4fd6a5...,target,8,152.50,10,326.40
2,ffee1b5d2c53a4808115bbfd35c296349e42dad9f94d19...,control,8,164.09,9,255.09
3,ffe7dce5da9cec7a57f459f85521fc6fcb1327fb179495...,target,9,257.50,9,257.50
4,ffcd57ea557d5182024e15e9f32b02e9b325059397bda2...,target,6,98.00,6,98.00


In [39]:
frequency_table = pd.crosstab(df_decil['decil_m12'], df_decil['decil_m1'])
frequency_table

decil_m1,1,2,3,4,5,6,7,8,9,10
decil_m12,,,,,,,,,,
1,218,11,21,42,33,45,26,10,12,1
2,0,188,2,35,43,45,42,25,22,11
3,0,0,191,15,43,64,42,41,26,11
4,0,0,0,178,31,55,60,35,46,20
5,0,0,0,0,164,40,75,68,62,31
6,0,0,0,0,0,153,59,85,102,46
7,0,0,0,0,0,0,137,67,139,119
8,0,0,0,0,0,0,0,136,154,191
9,0,0,0,0,0,0,0,0,145,366


In [ ]:
publico_janeiro_dezembro.head()

In [ ]:
publico_janeiro_dezembro[publico_janeiro_dezembro['order_created_month']==12]['decil'].unique()

In [ ]:
save_parquet(df_pivot, "gold", "df_decil.parquet")

In [ ]:
decil_summary=gerar_resumo_decis(df_decil)
decil_summary

In [ ]:
os.makedirs('../Resultados', exist_ok=True)

decil_summary.to_csv('../Resultados/decil_summary.csv', index=False)



Numero de pedidos em Janeiro e Dezembro para o quem esteve em ambos meses

In [40]:

pedidos_hist=resumo_coorte_ativa(publico_janeiro_dezembro,mes_coorte_inicio=12,mes_coorte_fim=1)
pedidos_hist

,is_target,Total_Pedidos_Mes_1,Total_Pedidos_Mes_12,Total_Clientes
0,control,1,1,382
1,control,2,1,131
2,control,2,2,139
3,control,3,1,91
4,control,3,2,72
...,...,...,...,...
358,target,56,24,1
359,target,62,33,1
360,target,84,21,1
361,target,102,38,1


In [ ]:
pedidos_hist.to_csv('../Resultados/pedidos_hist.csv', index=False)

Correcao decil